In [1]:
import pennylane as qml
import numpy as np
from pennylane import numpy as pnp
from matplotlib import pyplot as plt
from pennylane.operation import Operation, AnyWires
import os
import pandas as pd
import scipy.special as sp
import math
from scipy.stats import uniform_direction
from scipy.linalg import logm, svd

num_qubits = 5

# Initialize the device
dev = qml.device("default.qubit", wires=num_qubits)


In [2]:
# Construct the Hamiltonian terms
Hamiltonian_terms = []

# Interaction terms: XiX(i+1) + YiY(i+1) + ZiZ(i+1)
for i in range(num_qubits):
    Hamiltonian_terms.append(1.0 * (qml.PauliX(i) @ qml.PauliX((i+1)%num_qubits)) +
                                    (qml.PauliY(i) @ qml.PauliY((i+1)%num_qubits)) 
                                    + (qml.PauliZ(i) @ qml.PauliZ((i+1)%num_qubits)))

# Magnetic field terms: hZi
for i in range(num_qubits):
    Hamiltonian_terms.append(1.0 * qml.PauliZ(i))

# Define the Hamiltonian
Hamiltonian_operator = qml.Hamiltonian(coeffs=[1] * len(Hamiltonian_terms), observables=Hamiltonian_terms)

In [3]:
Hamiltonian = qml.Hamiltonian(coeffs=[1] * len(Hamiltonian_terms), observables=Hamiltonian_terms)

In [4]:


class fqs_op(qml.operation.Operation):
    num_params = 1
    num_wires = qml.operation.AnyWires
    par_domain = "R"

    @staticmethod
    def compute_matrix(axis):  # theta is the rotation angle
        """Custom operation for free-axis rotation"""
        q0, q1, q2, q3 = axis
        H = qml.sum(q0 * qml.Identity(AnyWires),  - 1j * q1 * qml.X(AnyWires), -1j * q2 * qml.Y(AnyWires), -1j * q3 * qml.Z(AnyWires))
        
        return H.matrix()

In [5]:

def entangling_layer_ladderZ(num_qubits):
    m = 0
    n = 1
    while m+1 < num_qubits:
        qml.CZ(wires=[m,m+1])
        m+=2
    
    while n+1 < num_qubits:
        qml.CZ(wires=[n,n+1])
        n+=2


@qml.qnode(dev)
def circuit(n_vectors, num_layers):
    """Parameterized quantum circuit with free-axis rotations"""
    
    for j in range(num_layers):
        for k in range(num_qubits):
            fqs_op(n_vectors[k + num_qubits * j], wires = k)
    
        entangling_layer_ladderZ(num_qubits)

    return qml.expval(Hamiltonian)

@qml.qnode(dev)
def circuit_state(n_vectors, num_layers, d, gate_type):
    """Parameterized quantum circuit with free-axis rotations"""
    
    ind = 0

    for j in range(num_layers):
    
        for k in range(num_qubits):

            if ind == d:
                if gate_type == "X":
                    fqs_op([0,1,0,0], wires=k)

                elif gate_type == "Y":
                    fqs_op([0,0,1,0], wires=k)

                elif gate_type == "Z":
                    fqs_op([0,0,0,1], wires=k)

                elif gate_type == "XY":
                    fqs_op([0, 1/np.sqrt(2), 1/np.sqrt(2), 0], wires=k)
   
                elif gate_type == "XZ":
                    fqs_op([0, 1/np.sqrt(2), 0, 1/np.sqrt(2)], wires=k)

                elif gate_type == "YZ":
                    fqs_op([0, 0, 1/np.sqrt(2), 1/np.sqrt(2)], wires=k)
                
                elif gate_type == "I":
                    fqs_op([1,0,0,0], wires=k)

                elif gate_type == "I_X":
                    fqs_op([1/np.sqrt(2), 1/np.sqrt(2), 0, 0], wires=k)

                elif gate_type == "I_Y":
                    fqs_op([1/np.sqrt(2),0, 1/np.sqrt(2), 0], wires=k)
                
                elif gate_type == "I_Z":
                    fqs_op([1/np.sqrt(2),0,0, 1/np.sqrt(2)], wires=k)

            else:
                fqs_op(n_vectors[k + num_qubits * j], wires = k)

            ind += 1
    
        entangling_layer_ladderZ(num_qubits)

    
    return qml.expval(Hamiltonian)


In [6]:

def compute_fqs_matrix(n_vectors, num_layers, d):
    """Compute the fqs matrix for a specific gate d"""

    rx = circuit_state(n_vectors, num_layers, d, gate_type="X")
    ry = circuit_state(n_vectors, num_layers, d, gate_type="Y")
    rz = circuit_state(n_vectors, num_layers, d, gate_type="Z")
    
    rxy = circuit_state(n_vectors, num_layers, d, gate_type="XY")
    rxz = circuit_state(n_vectors, num_layers, d, gate_type="XZ")
    ryz = circuit_state(n_vectors, num_layers, d, gate_type="YZ")
    
    Id = circuit_state(n_vectors, num_layers, d, gate_type="I")
    Id_x = circuit_state(n_vectors, num_layers, d, gate_type="I_X")
    Id_y = circuit_state(n_vectors, num_layers, d, gate_type="I_Y")
    Id_z = circuit_state(n_vectors, num_layers, d, gate_type="I_Z")
    
    #print(Id_z)


    matrix = [[Id             , Id_x-rx/2-Id/2, Id_y-ry/2-Id/2, Id_z-rz/2-Id/2],
                [Id_x-rx/2-Id/2,  rx            , (2*rxy-rx-ry)/2, (2*rxz-rx-rz)/2],
                [Id_y-ry/2-Id/2, (2*rxy-rx-ry)/2,  ry            , (2*ryz-ry-rz)/2],
                [Id_z-rz/2-Id/2, (2*rxz-rx-rz)/2, (2*ryz-ry-rz)/2,  rz            ]]

    return matrix



In [7]:
Id = np.matrix([[1,0],
               [0,1]])

X = np.matrix([[0,1],
               [1,0]])

Y = np.matrix([[0,-1j],
               [1j,0]])

Z = np.matrix([[1,0],
               [0,-1]])

In [8]:

def quaternion_to_unitary(q):
    U = q[0] * Id - 1j * q[1] * X - 1j * q[2] * Y - 1j * q[3] * Z
    return U


def is_unitary(U):
    return np.allclose(U.conj().T @ U, np.eye(U.shape[0]))

def spectral_norm_distance(U, V):
    diff = U - V
    return svd(diff, compute_uv=False)[0]  # Largest singular value

def frobenius_distance(U, V):
    diff = U - V
    return np.linalg.norm(diff, 'fro')

def bloch_dist(U, V):
    inner_product = np.trace(U.conj().T @ V)
    #angle = np.arccos(np.clip(np.real(inner_product) / 2, -1.0, 1.0))
    dist = np.sqrt(2.0 - np.abs(inner_product))
    
    return dist




In [9]:


def free_quaternion_selection(n_vectors, num_layers, iters, freeze_threshold, freeze_iters_k):
    """Implement the Fraxis algorithm"""

    num_gates = len(n_vectors)

    all_vals = []

    freeze_counters = np.zeros(len(n_vectors))    
    
    gate_opts_tresh = num_qubits * num_layers * iters 
    gate_opts = 0
    
    while True:

        if gate_opts > gate_opts_tresh:
            break

        for d in range(num_gates):
            
            if gate_opts > gate_opts_tresh:
                break
            
            if freeze_counters[d] > 0:
                freeze_counters[d] = freeze_counters[d] - 1
                #print(d)
                continue

            prev_q = np.array(n_vectors[d].copy())

            current_val = circuit(n_vectors, num_layers)

            # Use this for ideal simulator, FQS has some errors with ideal Quantum device
            if len(all_vals) > 0:
                if current_val > all_vals[-1] or current_val < min(Hamiltonian.eigvals()):
                    all_vals.append(all_vals[-1])
                else:
                    all_vals.append(current_val)
            else:
                all_vals.append(current_val)
            
            fqs_matrix = compute_fqs_matrix(n_vectors, num_layers, d)        

            eigVal, eigVec = np.linalg.eig(fqs_matrix)
            eigVec = np.transpose(eigVec)

            sid = np.argmin(eigVal)
            expected_val = np.amin(eigVal)
            
            if expected_val < current_val:
                n_vectors[d]  = eigVec[sid]

            current_q = np.array(n_vectors[d].copy())

            prev_unitary =  quaternion_to_unitary(prev_q)
            current_unitary = quaternion_to_unitary(current_q)

            bloch_dist_normalized = bloch_dist(prev_unitary, current_unitary) / np.sqrt(2.0)

            if (bloch_dist_normalized < freeze_threshold):
                freeze_iters_k[d] += 1
                
            gate_opts += 1

    return n_vectors, all_vals, freeze_iters_k



In [12]:
layers = [2]
n_iters = 2
runs = 4

# Pass freezing threshold as a list i.e. [0.01, 0.001]
Tvals = [0.01]

# sample from uniform distribution
uniform_sphere_sampler = uniform_direction(4)

for T in Tvals:
    freeze_thres =  T

    run_vals = []

    for layer in layers:
        
        print(f"Layers L={layer}")

        for j in range(runs):
            print("run: ", j+1)

            num_gates = num_qubits * layer
            freeze_iters = np.ones(num_gates)

            n_vectors = uniform_sphere_sampler.rvs(num_gates)
            
            optimal_n_vectors, energy_vals, freeze_iters_k = free_quaternion_selection(n_vectors, layer, n_iters, freeze_thres, freeze_iters)

            run_vals.append(energy_vals)

        file = f"MatrixNorm_1DHeisenberg_{num_qubits}Q_FQS_GateFreeze_T{T}_FreezeIterInc_{n_iters}cycles_{layer}layers_{runs}runs.xlsx"      
        
        columns = [f"run{i}" for i in range(1, runs+1)]
            
        run_vals = np.array(run_vals).T
        
        df = pd.DataFrame(run_vals, columns=columns)
        df.to_excel(file,  index=False)    
                



Layers L=2
run:  1
run:  2
run:  3
run:  4
